# **Q6**
Implement multiclass LOR, multiclass LOR with L2-norm regularization, and
multiclass LOR with L1-norm regularization models using BGD, SGD, and MBGD
algorithms. The multiclass extension of the LOR models must be done using One vs.
one and one vs. All coding algorithms. The dataset in data_q6_q7.txt contains 7
features and one output. The output is classified as class 1, class2, or class 3. You
must use hold-out cross-validation ((CV) with 70% as training, 10% as validation and
20% as testing) for the evaluation of training, validation, and testing instances for each
model. Evaluate the performance of each model using individual accuracy and overall
accuracy measures.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving data_q6_q7.txt to data_q6_q7.txt


In [ ]:
def hyp(x,w):
  return np.dot(x,w)
def hyp_lor(x,w):
  return 1/(1+np.exp(-(hyp(x,w))))
def mse(y,pred,l):
  return (1/l)*np.sum(np.square(y-pred))

In [ ]:
f,tr,va,te=8,147,21,42
data=pd.read_csv("/content/data_q6_q7.txt", delimiter = "\t", header=None)
data=pd.DataFrame(data)
data.columns=['a','b','c','d','e','f','g','class_name']
data.insert(0,"x0",1)
data.insert(f,"class_1",0);data.insert(f+1,"class_2",0);data.insert(f+2,"class_3",0)
dbool1,dbool2,dbool3 = (data.class_name==1),(data.class_name==2),(data.class_name==3)
data.loc[dbool1,'class_1'],data.loc[dbool2,'class_2'],data.loc[dbool3,'class_3'] = 1,1,1
data

# one vs all

In [ ]:
data=np.array(data); np.random.shuffle(data);np.random.shuffle(data)
# so far : boolean added for binary class; data shuffled
data_tr = data[:147,:-1].copy()
data_va = data[147:168,:-1].copy()
data_te = data[168:,:].copy()
norm = np.amax(np.abs(data_tr),axis=0)
data_tr,data_va,data_te_nor = data_tr/norm,data_va/norm,data_te[:,:-1]/norm #data normalised
# so far 2 : data split
x_test,y_test = data_te_nor[:,:f],data_te[:,f+3].reshape(te,1)
x_valid,y_valid = data_va[:,:f],data_va[:,f].reshape(va,1)
x_train = data_tr[:,:f]

y1_train = data_tr[:,f].reshape(tr,1)
y2_train = data_tr[:,f+1].reshape(tr,1)
y3_train = data_tr[:,f+2].reshape(tr,1)

## BGD

### LoR

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w1)-y1_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w1[j] = w1[j] - (1/tr)*2*jsum

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w2)-y2_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w2[j] = w2[j] - (1/tr)*0.2*jsum

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w3)-y3_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w3[j] = w3[j] - (1/tr)*1*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

Class 1 Accuracy = 0.8125
Class 2 Accuracy = 0.9
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9047619047619048


### LoR with L2 NORM

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w1)-y1_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w1[j] = (1-(0.001*1.25))*w1[j] - (1/tr)*1.25*jsum

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w2)-y2_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w2[j] = (1-(0.01*0.3))*w2[j] - (1/tr)*0.3*jsum

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w3)-y3_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w3[j] = (1-(0.01*0.3))*w3[j] - (1/tr)*0.3*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

Class 1 Accuracy = 0.8125
Class 2 Accuracy = 0.9
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9047619047619048


### LoR with L1 NORM

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w1)-y1_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w1[j] = w1[j] - (1/tr)*0.1*jsum - ((0.1*0.05)/2)*np.sign(w1[j])

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w2)-y2_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w2[j] = w2[j] - (1/tr)*1*jsum - ((1*0.005)/2)*np.sign(w2[j])

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x_train,w3)-y3_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x_train[:,j].reshape(tr,1)))
    w3[j] = w3[j] - (1/tr)*0.1*jsum - ((0.1*0.05)/2)*np.sign(w3[j])

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

Class 1 Accuracy = 0.8125
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286


## SGD

### LoR

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(1000):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w1)[index]-y1_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w1[j] = w1[j] - 0.3*j2

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(1000):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w2)[index]-y2_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w2[j] = w2[j] - 0.7*j2

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(1000):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w3)[index]-y3_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w3[j] = w3[j] - 0.7*j2

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[14.  0.  2.]
 [ 1.  9.  0.]
 [ 0.  0. 16.]]
Class 1 Accuracy = 0.875
Class 2 Accuracy = 0.9
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286


### LoR with L2 NORM

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w1)[index]-y1_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w1[j] = (1-(0.0004*1.3))*w1[j] - 1.3*j2

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w2)[index]-y2_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w2[j] = (1-(0.0009*1.2))*w2[j] - 1.2*j2

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w3)[index]-y3_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w3[j] = (1-(0.008*2))*w3[j] - 2*j2

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[17.  1.  0.]
 [ 1. 11.  0.]
 [ 6.  0.  6.]]
Class 1 Accuracy = 0.9444444444444444
Class 2 Accuracy = 0.9166666666666666
Class 3 Accuracy = 0.5
Overall Accuracy = 0.8095238095238095


### LoR with L1 NORM

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w1)[index]-y1_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w1[j] = w1[j] - 1.5*j2 - ((1.5*0.0005)/2)*np.sign(w1[j])

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w2)[index]-y2_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w2[j] = w2[j] - 1.3*j2 - ((1.3*0.0005)/2)*np.sign(w2[j])

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(tr)
  j1 = hyp_lor(x_train,w3)[index]-y3_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x_train[index,j].reshape(1,1))
    w3[j] = w3[j] - 1.2*j2 - ((1.2*0.0005)/2)*np.sign(w3[j])

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[16.  2.  0.]
 [ 1. 11.  0.]
 [ 4.  0.  8.]]
Class 1 Accuracy = 0.8888888888888888
Class 2 Accuracy = 0.9166666666666666
Class 3 Accuracy = 0.6666666666666666
Overall Accuracy = 0.8333333333333334


## MBGD

### LoR

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w1) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w1[j] = w1[j] - (1/15)*1.5*jsum

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w2) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w2[j] = w2[j] - (1/15)*0.7*jsum

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w3) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w3[j] = w3[j] - (1/15)*0.8*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[14.  0.  2.]
 [ 1.  9.  0.]
 [ 0.  0. 16.]]
Class 1 Accuracy = 0.875
Class 2 Accuracy = 0.9
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286


### LoR with L2 NORM

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1)
for t in range(600):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w1) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w1[j] = (1-(0.001*0.2))*w1[j] - 0.2*jsum

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1)
for t in range(600):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w2) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w2[j] = (1-(0.008*1))*w2[j] - 1*jsum

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1)
for t in range(600):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w3) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w3[j] = (1-(0.008*0.01))*w3[j] - 0.01*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[14.  4.  0.]
 [ 0. 12.  0.]
 [ 4.  0.  8.]]
Class 1 Accuracy = 0.7777777777777778
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.6666666666666666
Overall Accuracy = 0.8095238095238095


### LoR with L1 NORM

In [ ]:
# 1 vs (2,3)

w1=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w1) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w1[j] = w1[j] - (1/15)*1*jsum - ((1*0.05)/2)*np.sign(w1[j])

# 2 vs (1,3)

w2=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w2) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w2[j] = w2[j] - (1/15)*0.8*jsum - ((0.8*0.005)/2)*np.sign(w2[j])

# 3 vs (1,2)

w3=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(data_tr)
  hypo1 = np.dot(data_tr[:15,:f].reshape(15,f),w3) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,data_tr[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,data_tr[:15,j].reshape(15,1)))
    w3[j] = w3[j] - (1/15)*0.5*jsum - ((0.5*0.005)/2)*np.sign(w3[j])
# testing

y_pred=np.concatenate((hyp_lor(x_test,w1),hyp_lor(x_test,w2),hyp_lor(x_test,w3)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  class_def=np.argmax(y_pred[i])
  output[i]=class_def+1

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[13.  1.  2.]
 [ 0. 10.  0.]
 [ 0.  0. 16.]]
Class 1 Accuracy = 0.8125
Class 2 Accuracy = 1.0
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286


# one vs one

### data splitting

In [ ]:
data=np.array(data); np.random.shuffle(data);np.random.shuffle(data)
x_train = data[:147,:].copy() ; y_train = data[:147,-1].copy()
norm = np.amax(np.abs(x_train[:,:f]),axis=0)
x_test = data[147:189,:f].copy()/norm ; y_test = data[147:189,-1].copy().reshape(42,1) ; te=42
class_1 = x_train[np.where(y_train==1)]
class_2 = x_train[np.where(y_train==2)]
class_3 = x_train[np.where(y_train==3)]
train12 = np.concatenate((class_1,class_2),axis=0) ; np.random.shuffle(train12)
train23 = np.concatenate((class_2,class_3),axis=0) ; np.random.shuffle(train23)
train31 = np.concatenate((class_3,class_1),axis=0) ; np.random.shuffle(train31)
y12_train = train12[:,f].reshape(len(train12[:,f]),1)
y23_train = train23[:,f+1].reshape(len(train23[:,f+1]),1)
y31_train = train31[:,f+2].reshape(len(train31[:,f+2]),1)
x12_train = train12[:,:f].copy()/norm
x23_train = train23[:,:f].copy()/norm
x31_train = train31[:,:f].copy()/norm
norm_mb = list(norm) ; norm_mb = norm_mb+[1,1,1,1] ; norm_mb = np.array(norm_mb)

## LoR

### BGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x12_train,w12)-y12_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x12_train[:,j].reshape(len(y12_train),1)))
    w12[j] = w12[j] - (1/len(y12_train))*0.3*jsum

# 2 vs 3

w23=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x23_train,w23)-y23_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x23_train[:,j].reshape(len(y23_train),1)))
    w23[j] = w23[j] - (1/len(y23_train))*0.3*jsum

# 3 vs 1
w31=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x31_train,w31)-y31_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x31_train[:,j].reshape(len(y31_train),1)))
    w31[j] = w31[j] - (1/len(y31_train))*0.8*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

Class 1 Accuracy = 0.8461538461538461
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.9285714285714286


### SGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(len(y12_train))
  j1 = hyp_lor(x12_train,w12)[index]-y12_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x12_train[index,j].reshape(1,1))
    w12[j] = w12[j] - 0.3*j2

# 2 vs 1

w23=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(len(y23_train))
  j1 = hyp_lor(x23_train,w23)[index]-y23_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x23_train[index,j].reshape(1,1))
    w23[j] = w23[j] - 0.7*j2

# 3 vs 1

w31=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(len(y31_train))
  j1 = hyp_lor(x31_train,w31)[index]-y31_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x31_train[index,j].reshape(1,1))
    w31[j] = w31[j] - 0.7*j2

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[14.  1.  0.]
 [ 1. 11.  0.]
 [ 1.  0. 14.]]
Class 1 Accuracy = 0.9333333333333333
Class 2 Accuracy = 0.9166666666666666
Class 3 Accuracy = 0.9333333333333333
Overall Accuracy = 0.9285714285714286


### MBGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train12)
  tr12 = train12/norm_mb
  hypo1 = np.dot(tr12[:15,:f].reshape(15,f),w12) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr12[:15,f].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr12[:15,j].reshape(15,1)))
    w12[j] = w12[j] - (1/15)*0.6*jsum

# 2 vs 3

w23=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train23)
  tr23 = train23/norm_mb
  hypo1 = np.dot(tr23[:15,:f].reshape(15,f),w23) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr23[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr23[:15,j].reshape(15,1)))
    w23[j] = w23[j] - (1/15)*0.7*jsum

# 3 vs 1

w31=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train31)
  tr31 = train31/norm_mb
  hypo1 = np.dot(tr31[:15,:f].reshape(15,f),w31) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr31[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr31[:15,j].reshape(15,1)))
    w31[j] = w31[j] - (1/15)*0.8*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[14.  1.  0.]
 [ 1. 11.  0.]
 [ 0.  0. 15.]]
Class 1 Accuracy = 0.9333333333333333
Class 2 Accuracy = 0.9166666666666666
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9523809523809523


## LoR with L2 NORM

### BGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x12_train,w12)-y12_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x12_train[:,j].reshape(len(y12_train),1)))
    w12[j] = (1-(0.001*2))*w12[j] - (1/len(y12_train))*2*jsum

# 2 vs 3

w23=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x23_train,w23)-y23_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x23_train[:,j].reshape(len(y23_train),1)))
    w23[j] = (1-(0.002*1))*w23[j] - (1/len(y23_train))*1*jsum

# 3 vs 1
w31=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x31_train,w31)-y31_train # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x31_train[:,j].reshape(len(y31_train),1)))
    w31[j] = (1-(0.002*1))*w31[j] - (1/len(y31_train))*1*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

Class 1 Accuracy = 0.8571428571428571
Class 2 Accuracy = 0.9090909090909091
Class 3 Accuracy = 0.9411764705882353
Overall Accuracy = 0.9047619047619048


### SGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(600):
  index=np.random.randint(len(y12_train))
  j1 = hyp_lor(x12_train,w12)[index]-y12_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x12_train[index,j].reshape(1,1))
    w12[j] = (1-(0.0001*1))*w12[j] - 1*j2

# 2 vs 1

w23=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(len(y23_train))
  j1 = hyp_lor(x23_train,w23)[index]-y23_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x23_train[index,j].reshape(1,1))
    w23[j] = (1-(0.0001*1))*w23[j] - 1*j2

# 3 vs 1

w31=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  index=np.random.randint(len(y31_train))
  j1 = hyp_lor(x31_train,w31)[index]-y31_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x31_train[index,j].reshape(1,1))
    w31[j] = (1-(0.0001*1))*w31[j] - 1*j2
# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[13.  0.  1.]
 [ 2.  9.  0.]
 [ 1.  0. 16.]]
Class 1 Accuracy = 0.9285714285714286
Class 2 Accuracy = 0.8181818181818182
Class 3 Accuracy = 0.9411764705882353
Overall Accuracy = 0.9047619047619048


### MBGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train12)
  tr12 = train12/norm_mb
  hypo1 = np.dot(tr12[:5,:f].reshape(5,f),w12) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr12[:5,f].reshape(5,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr12[:5,j].reshape(5,1)))
    w12[j] = (1-(0.002*1))*w12[j] - (1/5)*1*jsum

# 2 vs 3

w23=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train23)
  tr23 = train23/norm_mb
  hypo1 = np.dot(tr23[:5,:f].reshape(5,f),w23) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr23[:5,f+1].reshape(5,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr23[:5,j].reshape(5,1)))
    w23[j] = (1-(0.002*1))*w23[j] - (1/5)*1*jsum

# 3 vs 1

w31=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train31)
  tr31 = train31/norm_mb
  hypo1 = np.dot(tr31[:5,:f].reshape(5,f),w31) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr31[:5,f+2].reshape(5,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr31[:5,j].reshape(5,1)))
    w31[j] = (1-(0.002*1))*w31[j] - (1/5)*1*jsum

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[ 9.  1.  2.]
 [ 0. 22.  0.]
 [ 1.  0.  7.]]
Class 1 Accuracy = 0.75
Class 2 Accuracy = 1.0
Class 3 Accuracy = 0.875
Overall Accuracy = 0.9047619047619048


## LoR with L1 NORM

### BGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x12_train,w12)-y12_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x12_train[:,j].reshape(len(y12_train),1)))
    w12[j] = w12[j] - (1/len(y12_train))*2*jsum - ((2*0.0005)/2)*np.sign(w12[j])

# 2 vs 3

w23=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x23_train,w23)-y23_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x23_train[:,j].reshape(len(y23_train),1)))
    w23[j] = w23[j] - (1/len(y23_train))*0.3*jsum - ((0.3*0.005)/2)*np.sign(w23[j])

# 3 vs 1
w31=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(500):
  j1 = hyp_lor(x31_train,w31)-y31_train, # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,x31_train[:,j].reshape(len(y31_train),1)))
    w31[j] = w31[j] - (1/len(y31_train))*0.8*jsum - ((0.8*0.005)/2)*np.sign(w31[j])

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

Class 1 Accuracy = 0.875
Class 2 Accuracy = 0.9166666666666666
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286


### SGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(1000):
  index=np.random.randint(len(y12_train))
  j1 = hyp_lor(x12_train,w12)[index]-y12_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x12_train[index,j].reshape(1,1))
    w12[j] = w12[j] - 0.7*j2 - ((0.7*0.005)/2)*np.sign(w12[j])

# 2 vs 1

w23=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(1000):
  index=np.random.randint(len(y23_train))
  j1 = hyp_lor(x23_train,w23)[index]-y23_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x23_train[index,j].reshape(1,1))
    w23[j] = w23[j] - 0.7*j2 - ((0.7*0.0005)/2)*np.sign(w23[j])

# 3 vs 1

w31=np.random.rand(f).reshape(f,1) # weight vector initialised
for t in range(1000):
  index=np.random.randint(len(y31_train))
  j1 = hyp_lor(x31_train,w31)[index]-y31_train[index] # (hyp(xi)-yi)
  for j in range(f):
    j2 = np.multiply(j1,x31_train[index,j].reshape(1,1))
    w31[j] = w31[j] - 0.7*j2 - ((0.7*0.005)/2)*np.sign(w31[j])

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[15.  0.  1.]
 [ 3.  9.  0.]
 [ 0.  0. 14.]]
Class 1 Accuracy = 0.9375
Class 2 Accuracy = 0.75
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9047619047619048


### MBGD

In [ ]:
# 1 vs 2

w12=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train12)
  tr12 = train12/norm_mb
  hypo1 = np.dot(tr12[:15,:f].reshape(15,f),w12) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr12[:15,f].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr12[:15,j].reshape(15,1)))
    w12[j] = w12[j] - (1/15)*0.6*jsum - ((0.6*0.005)/2)*np.sign(w12[j])

# 2 vs 3

w23=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train23)
  tr23 = train23/norm_mb
  hypo1 = np.dot(tr23[:15,:f].reshape(15,f),w23) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr23[:15,f+1].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr23[:15,j].reshape(15,1)))
    w23[j] = w23[j] - (1/15)*0.7*jsum - ((0.7*0.0025)/2)*np.sign(w23[j])

# 3 vs 1

w31=np.random.rand(f).reshape(f,1)
for t in range(450):
  np.random.shuffle(train31)
  tr31 = train31/norm_mb
  hypo1 = np.dot(tr31[:15,:f].reshape(15,f),w31) # hypothesis array part1
  hypo = 1/(1+np.exp(-hypo1))
  j1 = np.subtract(hypo,tr31[:15,f+2].reshape(15,1)) # (hyp(xi)-yi)
  for j in range(f):
    jsum = np.sum(np.multiply(j1,tr31[:15,j].reshape(15,1)))
    w31[j] = w31[j] - (1/15)*0.8*jsum - ((0.8*0.005)/2)*np.sign(w31[j])

# testing

y_pred=np.concatenate((hyp_lor(x_test,w12),hyp_lor(x_test,w23),hyp_lor(x_test,w31)),axis=1)
output=np.zeros((te,1))

for i in range(te):
  y_pred[i][0] = 1 if (y_pred[i][0]>0.5) else 2
  y_pred[i][1] = 2 if (y_pred[i][1]>0.5) else 3
  y_pred[i][2] = 3 if (y_pred[i][2]>0.5) else 1
  class_def=stats.mode(y_pred[i])
  output[i]=class_def[0]

comparison=np.concatenate((output,y_test),axis=1)

# performance evaluation

conf_matrix=np.zeros((3,3)) # confusion matrix
for comp in comparison:
  pred,act=int(comp[0]-1),int(comp[1]-1)
  conf_matrix[act][pred]+=1
print(conf_matrix)

cm,o_num,o_den=conf_matrix,0,0
for i in range(3):
  den=0
  for j in range(3):
    den=den+cm[i][j]
  acc=cm[i][i]/den
  print('Class {} Accuracy = {}'.format(i+1,acc))
  o_num=o_num+cm[i][i]
  o_den=o_den+den

print('Overall Accuracy = {}'.format(o_num/o_den))

[[14.  0.  2.]
 [ 1. 11.  0.]
 [ 0.  0. 14.]]
Class 1 Accuracy = 0.875
Class 2 Accuracy = 0.9166666666666666
Class 3 Accuracy = 1.0
Overall Accuracy = 0.9285714285714286
